In [8]:
import pandas as pd
import math
import datetime
import statistics
from pyFTS.common import Util
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
import numpy as np
import matplotlib.pyplot as plt
from pyFTS.benchmarks import Measures

In [9]:
def cal_nrmse(rmse, y):
    x = max(y)-min(y)
    return (rmse/x)

In [35]:
df = pd.read_csv('/home/hugo/projetos-doutorado/Appliance-Energy-Prediction/data/HomeC.csv')
data = df.drop(labels=['icon','summary','cloudCover', 'House overall [kW]', 'time'], axis=1)
#data = data.loc[0: : 10]
#data['use [kW]'] = data['use [kW]'] * 1000; # transforma para Watt
data.dropna()

/home/hugo/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (0,27) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,use [kW],gen [kW],Dishwasher [kW],Furnace 1 [kW],Furnace 2 [kW],Home office [kW],Fridge [kW],Wine cellar [kW],Garage door [kW],Kitchen 12 [kW],...,temperature,humidity,visibility,apparentTemperature,pressure,windSpeed,windBearing,precipIntensity,dewPoint,precipProbability
0,0.932833,0.003483,0.000033,0.020700,0.061917,0.442633,0.124150,0.006983,0.013083,0.000417,...,36.14,0.62,10.00,29.26,1016.91,9.18,282.0,0.0000,24.40,0.00
1,0.934333,0.003467,0.000000,0.020717,0.063817,0.444067,0.124000,0.006983,0.013117,0.000417,...,36.14,0.62,10.00,29.26,1016.91,9.18,282.0,0.0000,24.40,0.00
2,0.931817,0.003467,0.000017,0.020700,0.062317,0.446067,0.123533,0.006983,0.013083,0.000433,...,36.14,0.62,10.00,29.26,1016.91,9.18,282.0,0.0000,24.40,0.00
3,1.022050,0.003483,0.000017,0.106900,0.068517,0.446583,0.123133,0.006983,0.013000,0.000433,...,36.14,0.62,10.00,29.26,1016.91,9.18,282.0,0.0000,24.40,0.00
4,1.139400,0.003467,0.000133,0.236933,0.063983,0.446533,0.122850,0.006850,0.012783,0.000450,...,36.14,0.62,10.00,29.26,1016.91,9.18,282.0,0.0000,24.40,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
503905,1.601233,0.003183,0.000050,0.085267,0.642417,0.041783,0.005267,0.008667,0.013483,0.000467,...,35.12,0.86,8.74,29.45,1011.49,6.72,186.0,0.0101,31.27,0.51
503906,1.599333,0.003233,0.000050,0.104017,0.625033,0.041750,0.005233,0.008433,0.013433,0.000467,...,35.12,0.86,8.74,29.45,1011.49,6.72,186.0,0.0101,31.27,0.51
503907,1.924267,0.003217,0.000033,0.422383,0.637733,0.042033,0.004983,0.008467,0.012933,0.000533,...,35.12,0.86,8.74,29.45,1011.49,6.72,186.0,0.0101,31.27,0.51
503908,1.978200,0.003217,0.000050,0.495667,0.620367,0.042100,0.005333,0.008233,0.012817,0.000517,...,35.12,0.86,8.74,29.45,1011.49,6.72,186.0,0.0101,31.27,0.51


In [36]:
# Create a lag feature
y_appliance = data['use [kW]']
var = pd.DataFrame(y_appliance)
dataframe = pd.concat([var.shift(1), var], axis=1)
dataframe.columns = ['t', 't+1']

In [37]:
dataframe

,t,t+1
0,NaN,0.932833
1,0.932833,0.934333
2,0.934333,0.931817
3,0.931817,1.022050
4,1.022050,1.139400
...,...,...
503906,1.601233,1.599333
503907,1.599333,1.924267
503908,1.924267,1.978200
503909,1.978200,1.990950


In [38]:
# Create a baseline model (Naive model)
def model_persistence(x):
  return x

def test_persistence(test_X):
    predictions = list()
    for x in test_X:
        yhat = model_persistence(x)
        predictions.append(yhat)
    return predictions

In [39]:
def cal_nrmse(rmse, y):
    x = max(y)-min(y)
    return (rmse/x)

In [40]:
result = {
     "window": [],
     "rmse": [],
     "nrmse": [],
     "mape": [],
     "mae": [],
     "smape": [],
     "nrmse":[]
}

tam = len(dataframe)
n_windows = 30
windows_length = math.floor(tam / n_windows)
for ct, ttrain, ttest in Util.sliding_window(dataframe[1:tam], windows_length, 0.75, inc=1):
    if len(ttest) > 0:
        
        print('-' * 20)
        print(f'training window {(ct)}')
        
        train_X, train_y = ttrain.iloc[:,0], ttrain.iloc[:,1]
        test_X, test_y = ttest.iloc[:,0], ttest.iloc[:,1]
        
        forecast = test_persistence(test_X)
        
        #print("[{0: %H:%M:%S}]".format(datetime.datetime.now()) + f" getting statistics")
        rmse = Measures.rmse(test_y,forecast)
        nrmse = cal_nrmse(rmse, test_y)
        mape = Measures.mape(test_y,forecast)
        smape = Measures.smape(test_y,forecast)
        mae = mean_absolute_error(test_y, forecast)
        r2 = r2_score(test_y, forecast)
        nrmse = cal_nrmse(rmse,test_y)
        
        result["rmse"].append(round(rmse,3))
        result["mape"].append(round(mape,3))
        result["smape"].append(round(smape,3))
        result["mae"].append(round(mae,3))
        result["nrmse"].append(round(nrmse,3))
        result["window"].append(ct)
        
measures = pd.DataFrame(result)

--------------------
training window 0
--------------------
training window 16797
--------------------
training window 33594
--------------------
training window 50391
--------------------
training window 67188
--------------------
training window 83985
--------------------
training window 100782
--------------------
training window 117579
--------------------
training window 134376
--------------------
training window 151173
--------------------
training window 167970
--------------------
training window 184767
--------------------
training window 201564
--------------------
training window 218361
--------------------
training window 235158
--------------------
training window 251955
--------------------
training window 268752
--------------------
training window 285549
--------------------
training window 302346
--------------------
training window 319143
--------------------
training window 335940
--------------------
training window 352737
--------------------
training window 36953

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [41]:
measures

,window,rmse,nrmse,mape,mae,smape
0,0,0.740,0.176,52.329,0.506,21.951
1,1679,0.702,0.171,86.803,0.522,26.000
2,3358,0.491,0.175,71.545,0.326,22.173
3,5037,0.633,0.126,51.579,0.472,19.038
4,6716,0.530,0.146,103.782,0.383,24.744
5,8395,0.370,0.139,42.884,0.189,16.595
6,10074,0.599,0.139,80.843,0.414,24.546
7,11753,0.486,0.142,31.912,0.228,13.136
8,13432,0.573,0.132,70.957,0.299,19.978
9,15111,0.465,0.082,40.792,0.212,15.641


In [42]:
measures.to_csv (r'win_persistence_ksh_1min.csv', index = False, header=True)

In [43]:
#measures.to_csv (r'win_persistence_ksh_10min.csv', index = False, header=True)

In [44]:

final_result = {
     "rmse": [],
     "nrmse": [],
     "mape": [],
     "mae": [],
     "smape": [],
     "nrmse":[],
     "rmse_std": [],
     "mae_std": [],
     "mape_std": [],
     "smape_std": [],
     "nrmse_std": []
}

final_result["rmse"].append(round(statistics.mean(measures['rmse']),3))
final_result["mape"].append(round(statistics.mean(measures['mape']),3))
final_result["smape"].append(round(statistics.mean(measures['smape']),3))
final_result["mae"].append(round(statistics.mean(measures['mae']),3))
final_result["nrmse"].append(round(statistics.mean(measures['nrmse']),3))

final_result["rmse_std"].append(round(statistics.stdev(measures['rmse']),3))
final_result["mape_std"].append(round(statistics.stdev(measures['mape']),3))
final_result["smape_std"].append(round(statistics.stdev(measures['smape']),3))
final_result["mae_std"].append(round(statistics.stdev(measures['mae']),3))
final_result["nrmse_std"].append(round(statistics.stdev(measures['nrmse']),3))
        
final_measures_persistence = pd.DataFrame(final_result)

print("Mean Statistics Persistence: ")
final_measures_persistence

Mean Statistics Persistence: 


,rmse,nrmse,mape,mae,smape,rmse_std,mae_std,mape_std,smape_std,nrmse_std
0,0.846,0.153,251.684,0.468,21.237,0.719,0.41,919.409,5.927,0.049


In [45]:
final_measures_persistence.to_csv (r'persistence_ksh_1min.csv', index = False, header=True)

In [21]:
#final_measures_persistence.to_csv (r'persistence_ksh_10min.csv', index = False, header=True)